In [1]:
#Written_By_Eng. Mohammed Albhaisi M.Sc.

from nuscenes import NuScenes

#Load ground truth data
nusc = NuScenes(version='v1.0-trainval', dataroot='/home/albhaisi/Desktop/WS/data', verbose=True)

Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 26.345 seconds.
Reverse indexing ...
Done reverse indexing in 6.0 seconds.


In [2]:
#Load_results

import json
from nuscenes.eval.tracking.data_classes import TrackingBox

#Give tracker result path to evaluate 
result_path = '/home/albhaisi/Desktop/senarios/final_results/3_straight_toward_ego/Pillar02.json'

#Give one tracking id from result
tracking_idd='car_2_32'


def load_predictions(result_path):
        #Choose which scene results to upload 
        with open(result_path) as f:
                data = json.load(f)
                result = data['results']
                #result['tracking_id']
                print(result)

        return result

prediction_results= load_predictions(result_path)
#print(prediction_results)

#Check_results
#print(prediction_results)

{'51f37263b06c417e94138f2d9348f909': [{'translation': [591.558445239331, 1419.969785298677, 1.5052879867416635], 'size': [1.8919219970703125, 4.492376327514648, 1.7408502101898193], 'velocity': [0.0, 0.0], 'rotation': [-0.6341851091461751, 0.0, 0.0, 0.7731812512841048], 'tracking_score': 0.737276554107666, 'sample_token': '51f37263b06c417e94138f2d9348f909', 'tracking_id': 'car_2_0', 'tracking_name': 'car'}, {'translation': [586.6092516974919, 1420.2385305068099, 2.052836909417894], 'size': [1.9731638431549072, 4.680844306945801, 1.8040544986724854], 'velocity': [0.0, 0.0], 'rotation': [0.7241846044333917, 0.0, 0.0, 0.6896061620241309], 'tracking_score': 0.42920613288879395, 'sample_token': '51f37263b06c417e94138f2d9348f909', 'tracking_id': 'car_2_1', 'tracking_name': 'car'}, {'translation': [584.2075297188079, 1440.121714494857, 1.9922553085943366], 'size': [1.980210781097412, 4.568724632263184, 1.7223302125930786], 'velocity': [0.0, 0.0], 'rotation': [0.7498167522019665, 0.0, 0.0, 0.6

In [3]:
from collections import defaultdict
from typing import List, Dict, Tuple

import numpy as np

from nuscenes.eval.common.data_classes import MetricData, EvalBox
from nuscenes.eval.common.utils import center_distance, scale_iou
from nuscenes.eval.detection.constants import DETECTION_NAMES, ATTRIBUTE_NAMES, TP_METRICS

class DetectionBox(EvalBox):
    """ Data class used during detection evaluation. Can be a prediction or ground truth."""

    def __init__(self,
                 sample_token: str = "",                 
                 sample_annotation_token: str = "", # sample_annotation_token only for gt boxes.
                 translation: Tuple[float, float, float] = (0, 0, 0),
                 size: Tuple[float, float, float] = (0, 0, 0),
                 rotation: Tuple[float, float, float, float] = (0, 0, 0, 0),
                 velocity: Tuple[float, float] = (0, 0),
                 ego_translation: [float, float, float] = (0, 0, 0),  # Translation to ego vehicle in meters.
                 num_pts: int = -1,  # Nbr. LIDAR or RADAR inside the box. Only for gt boxes.
                 tracking_name: str = 'car',  # The class name used in the detection challenge.
                 detection_name: str = 'car',
                 detection_score: float = -1.0,
                 tracking_score: float = -1.0):  # GT samples do not have a score.
                 
          

        super().__init__(sample_token, translation, size, rotation, velocity, ego_translation, num_pts)

        assert tracking_name is not None, 'Error: detection_name cannot be empty!'
        assert tracking_name in DETECTION_NAMES, 'Error: Unknown detection_name %s' % tracking_name


        assert type(tracking_score) == float, 'Error: detection_score must be a float!'
        assert not np.any(np.isnan(tracking_score)), 'Error: detection_score may not be NaN!'

        # Assign.
        self.tracking_name = tracking_name
        self.detection_name= detection_name
        self.tracking_score = tracking_score
        self.detection_score = detection_score
    def __eq__(self, other):
        return (self.sample_token == other.sample_token and
                self.sample_annotation_token == other.sample_annotation_token and
                self.translation == other.translation and
                self.size == other.size and
                self.rotation == other.rotation and
                self.velocity == other.velocity and
                self.ego_translation == other.ego_translation and
                self.num_pts == other.num_pts and
                self.tracking_name == other.tracking_name and
                self.detection_name == other.detection_name and
                self.detection_score == other.detection_score and
                self.tracking_score == other.tracking_score)

    def serialize(self) -> dict:
        """ Serialize instance into json-friendly format. """
        return {
            'sample_token': self.sample_token,
            'sample_annotation_token': self.sample_annotation_token,
            'translation': self.translation,
            'size': self.size,
            'rotation': self.rotation,
            'velocity': self.velocity,
            'ego_translation': self.ego_translation,
            'ego_dist': self.ego_dist, # return ego distance as well
            'num_pts': self.num_pts,
            'tracking_name': self.tracking_name,
            'detection_name': self.detection_name,
            'tracking_score': self.tracking_score,
            'detection_score': self.detection_score
    
        }

    @classmethod
    def deserialize(cls, content: dict):
        """ Initialize from serialized content. """
        return cls(sample_token=content['sample_token'],
                   sample_annotation_token= "" if 'sample_annotation_token' not in content
                   else content['sample_annotation_token'],
                   translation=tuple(content['translation']),
                   size=tuple(content['size']),
                   rotation=tuple(content['rotation']),
                   velocity=tuple(content['velocity']),
                   ego_translation=(0.0, 0.0, 0.0) if 'ego_translation' not in content
                   else tuple(content['ego_translation']),
                   num_pts=-1 if 'num_pts' not in content else int(content['num_pts']),
                   tracking_name=content['tracking_name'],
                   detection_name=content['tracking_name'],
                   detection_score=-1.0 if 'detection_score' not in content else float(content['tracking_score']),
                   tracking_score=-1.0 if 'tracking_score' not in content else float(content['tracking_score']))

In [4]:
#Load_ground_truth_for_one_scene

from typing import Dict, Tuple
import numpy as np
import tqdm
from pyquaternion import Quaternion
from nuscenes.eval.detection.utils import category_to_detection_name
from nuscenes.utils.data_classes import Box
from nuscenes.utils.geometry_utils import points_in_box
from nuscenes.utils.splits import create_splits_scenes
from nuscenes.eval.common.data_classes import EvalBoxes
from nuscenes.eval.tracking.data_classes import TrackingBox


eval_set='val'

def get_some_sample():
        uniqe_samples =[]
        #To know number of annotation for all instances
        for sample_ in prediction_results:
                pred_results = prediction_results[sample_]
                for trackers in pred_results:
                     track_id = trackers['tracking_id']
                     if track_id == tracking_idd:
                          uniqe_samples.append(sample_)
        return uniqe_samples
                          

                
                        
                     
print(get_some_sample())
eval_set: str
verbose: bool = True


def get_all_samples_in_scene():
        all_samples_in_scene =[]
        #To know number of annotation for all instances
        for samples in prediction_results:
                all_samples_in_scene.append(samples)
        return all_samples_in_scene

#print(get_all_samples_in_scene())

def load_gt(nusc: NuScenes, eval_split: str, box_cls, verbose: bool = False) -> EvalBoxes:
    """
    Loads ground truth boxes from DB.
    :param nusc: A NuScenes instance.
    :param eval_split: The evaluation split for which we load GT boxes.
    :param box_cls: Type of box to load, e.g. DetectionBox or TrackingBox.
    :param verbose: Whether to print messages to stdout.
    :return: The GT boxes.
    """
    # Init.
    if box_cls == DetectionBox:
        attribute_map = {a['token']: a['name'] for a in nusc.attribute}

    if verbose:
        print('Loading annotations for {} split from nuScenes version: {}'.format(eval_split, nusc.version))
    # Read out all sample_tokens in DB.
    sample_tokens_all = get_some_sample()
    assert len(sample_tokens_all) > 0, "Error: Database has no samples!"

    # Only keep samples from this split.
    splits = create_splits_scenes()

    # Check compatibility of split with nusc_version.
    version = nusc.version
    if eval_split in {'train', 'val', 'train_detect', 'train_track'}:
        assert version.endswith('trainval'), \
            'Error: Requested split {} which is not compatible with NuScenes version {}'.format(eval_split, version)
    elif eval_split in {'mini_train', 'mini_val'}:
        assert version.endswith('mini'), \
            'Error: Requested split {} which is not compatible with NuScenes version {}'.format(eval_split, version)
    elif eval_split == 'test':
        assert version.endswith('test'), \
            'Error: Requested split {} which is not compatible with NuScenes version {}'.format(eval_split, version)
    else:
        raise ValueError('Error: Requested split {} which this function cannot map to the correct NuScenes version.'
                         .format(eval_split))

    if eval_split == 'test':
        # Check that you aren't trying to cheat :).
        assert len(nusc.sample_annotation) > 0, \
            'Error: You are trying to evaluate on the test set but you do not have the annotations!'

    sample_tokens = []
    for sample_token in sample_tokens_all:
        scene_token = nusc.get('sample', sample_token)['scene_token']
        scene_record = nusc.get('scene', scene_token)
        if scene_record['name'] in splits[eval_split]:
            sample_tokens.append(sample_token)

    all_annotations = EvalBoxes()

    # Load annotations and filter predictions and annotations.
    tracking_id_set = set()
    for sample_token in tqdm.tqdm(sample_tokens, leave=verbose):

        sample = nusc.get('sample', sample_token)
        sample_annotation_tokens = sample['anns']

        sample_boxes = []
        for sample_annotation_token in sample_annotation_tokens:

            sample_annotation = nusc.get('sample_annotation', sample_annotation_token)
            if box_cls == DetectionBox:
                # Get label name in detection task and filter unused labels.
                tracking_name = category_to_detection_name(sample_annotation['category_name'])
                if tracking_name is None:
                    continue



                sample_boxes.append(
                    box_cls(
                        sample_token=sample_token,
                        translation=sample_annotation['translation'],
                        size=sample_annotation['size'],
                        rotation=sample_annotation['rotation'],
                        velocity=nusc.box_velocity(sample_annotation['token'])[:2],
                        num_pts=sample_annotation['num_lidar_pts'] + sample_annotation['num_radar_pts'],
                        tracking_name=tracking_name,
                        tracking_score=-1.0,  # GT samples do not have a score.
                    )
                )
            elif box_cls == TrackingBox:
                # Use nuScenes token as tracking id.
                tracking_id = sample_annotation['instance_token']
                tracking_id_set.add(tracking_id)

                # Get label name in detection task and filter unused labels.
                # Import locally to avoid errors when motmetrics package is not installed.
                from nuscenes.eval.tracking.utils import category_to_tracking_name
                tracking_name = category_to_tracking_name(sample_annotation['category_name'])
                if tracking_name is None:
                    continue

                sample_boxes.append(
                    box_cls(
                        sample_token=sample_token,
                        translation=sample_annotation['translation'],
                        size=sample_annotation['size'],
                        rotation=sample_annotation['rotation'],
                        velocity=nusc.box_velocity(sample_annotation['token'])[:2],
                        num_pts=sample_annotation['num_lidar_pts'] + sample_annotation['num_radar_pts'],
                        tracking_id=tracking_id,
                        tracking_name=tracking_name,
                        tracking_score=-1.0  # GT samples do not have a score.
                    )
                )
            else:
                raise NotImplementedError('Error: Invalid box_cls %s!' % box_cls)

        all_annotations.add_boxes(sample_token, sample_boxes)

    if verbose:
        print("Loaded ground truth annotations for {} samples.".format(len(all_annotations.sample_tokens)))

    return all_annotations

gt_boxes = load_gt(nusc, eval_set, TrackingBox, verbose=verbose)





['6a02dd6bc13a457b9af4dc88220e269e', 'd492d47f5aba43249fe426f07a2911bf', '7b175346a22b4e68b54a9d21f88446dc', '1192c1448ec044ef97df3078e92032eb', '6f82d20df0564605978ac7ee7d2da494', '2f207084f4794686be52b8fbbfa91f1f', '71a74ae092364afebe91caaee4ed2af3', 'a95deb56b3a845acae5cc782f42e5e4c', '76b0296bc28a454894ea616e547ee9e9', '8ccc7487ae0a4aed9daa021f6efe96b5', '91bb8353dea24e0eb219c013b45dcd64', 'adebba609dd34ac58da232e03ad80f89', 'f2727eb072f7470b862601edee7250b3', '849bde6d489544178c838baa28142bd4', 'afd9b98ede4b4c79b3fc1882ffeb2068', '12a72195b0a242b3bf7a48c0d82cd219', '12a0035c7d934d259f5fa32eafd7b0e3', 'eab9a7fe8c7c4cbebbefbbdb77960d6a', '000681a060c04755a1537cf83b53ba57', '39118d2a1a9b459eadc2c2d28c7658d3', 'b70d359986c74a008d7e9f957521b213', 'c1348c56187d4f38ad1b569cd3a7095f', 'e5062bd5d7774291b8641c5a9a38b439', 'cf6a7d894c034a4db251f41bc31cdbf7', '9de41b523cce4fb3aa4f6230ec06345d', '5bbe677b299143128aa0492add6b28cf', '8b2da12cffd84eb2925b0b9e08ed250f', 'd48f731eacf64f1f8df51ba2a1

100%|██████████| 31/31 [00:00<00:00, 87.86it/s]

Loaded ground truth annotations for 31 samples.


In [5]:
# nuScenes dev-kit.
# Code written by Holger Caesar & Oscar Beijbom, 2019.

import abc
from collections import defaultdict
from typing import List, Tuple, Union
import numpy as np

#EvaBox is edited to evaluate just one instance
class EvalBox(abc.ABC):
    """ Abstract base class for data classes used during detection evaluation. Can be a prediction or ground truth."""

    def __init__(self,
                 sample_token: str = "",
                 translation: Tuple[float, float, float] = (0, 0, 0),
                 size: Tuple[float, float, float] = (0, 0, 0),
                 rotation: Tuple[float, float, float, float] = (0, 0, 0, 0),
                 velocity: Tuple[float, float] = (0, 0),
                 ego_translation: Tuple[float, float, float] = (0, 0, 0),  # Translation to ego vehicle in meters.
                 num_pts: int = -1):  # Nbr. LIDAR or RADAR inside the box. Only for gt boxes.

        # Assert data for shape and NaNs.
        assert type(sample_token) == str, 'Error: sample_token must be a string!'

        assert len(translation) == 3, 'Error: Translation must have 3 elements!'
        assert not np.any(np.isnan(translation)), 'Error: Translation may not be NaN!'

        assert len(size) == 3, 'Error: Size must have 3 elements!'
        assert not np.any(np.isnan(size)), 'Error: Size may not be NaN!'

        assert len(rotation) == 4, 'Error: Rotation must have 4 elements!'
        assert not np.any(np.isnan(rotation)), 'Error: Rotation may not be NaN!'

        # Velocity can be NaN from our database for certain annotations.
        assert len(velocity) == 2, 'Error: Velocity must have 2 elements!'

        assert len(ego_translation) == 3, 'Error: Translation must have 3 elements!'
        assert not np.any(np.isnan(ego_translation)), 'Error: Translation may not be NaN!'

        assert type(num_pts) == int, 'Error: num_pts must be int!'
        assert not np.any(np.isnan(num_pts)), 'Error: num_pts may not be NaN!'

        # Assign.
        self.sample_token = sample_token
        self.translation = translation
        self.size = size
        self.rotation = rotation
        self.velocity = velocity
        self.ego_translation = ego_translation
        self.num_pts = num_pts

    @property
    def ego_dist(self) -> float:
        """ Compute the distance from this box to the ego vehicle in 2D. """
        return np.sqrt(np.sum(np.array(self.ego_translation[:2]) ** 2))

    def __repr__(self):
        return str(self.serialize())

    @abc.abstractmethod
    def serialize(self) -> dict:
        pass

    @classmethod
    @abc.abstractmethod
    def deserialize(cls, content: dict):
        pass


EvalBoxType = Union['DetectionBox', 'TrackingBox']


class EvalBoxes:
    """ Data class that groups EvalBox instances by sample. """

    def __init__(self):
        """
        Initializes the EvalBoxes for GT or predictions.
        """
        self.boxes = defaultdict(list)

    def __repr__(self):
        return "EvalBoxes with {} boxes across {} samples".format(len(self.all), len(self.sample_tokens))

    def __getitem__(self, item) -> List[EvalBoxType]:
        return self.boxes[item]

    def __eq__(self, other):
        if not set(self.sample_tokens) == set(other.sample_tokens):
            return False
        for token in self.sample_tokens:
            if not len(self[token]) == len(other[token]):
                return False
            for box1, box2 in zip(self[token], other[token]):
                if box1 != box2:
                    return False
        return True

    def __len__(self):
        return len(self.boxes)

    @property
    def all(self) -> List[EvalBoxType]:
        """ Returns all EvalBoxes in a list. """
        ab = []
        for sample_token in self.sample_tokens:
            ab.extend(self[sample_token])
        return ab

    @property
    def sample_tokens(self) -> List[str]:
        """ Returns a list of all keys. """
        return list(self.boxes.keys())

    def add_boxes(self, sample_token: str, boxes: List[EvalBoxType]) -> None:
        """ Adds a list of boxes. """
        self.boxes[sample_token].extend(boxes)

    def serialize(self) -> dict:
        """ Serialize instance into json-friendly format. """
        return {key: [box.serialize() for box in boxes] for key, boxes in self.boxes.items()}

    @classmethod
    def deserialize(cls, content: dict, box_cls):
        """
        Initialize from serialized content.
        :param content: A dictionary with the serialized content of the box.
        :param box_cls: The class of the boxes, DetectionBox or TrackingBox.
        """
        alist_test=[]
        eb = cls()
        for sample_token in content:
            pred_results = content[sample_token]
                #print(pred_results)
           
            for trackers in pred_results:
                                #print(pred_results)
                                track_id = trackers['tracking_id']
                                #print(track_id)
                                if track_id == tracking_idd:
                                        ourresult=trackers
                                        #print(ourresult)
                                        alist_test.append(ourresult)
                                        eb.add_boxes(sample_token, [box_cls.deserialize(box) for box in alist_test])
                                        
                                        
                                     
                                        #print(trackers['tracking_id'])
                                        #print(boxes)
        #print(alist_test)
            
            
        return eb


class MetricData(abc.ABC):
    """ Abstract base class for the *MetricData classes specific to each task. """

    @abc.abstractmethod
    def serialize(self):
        """ Serialize instance into json-friendly format. """
        pass

    @classmethod
    @abc.abstractmethod
    def deserialize(cls, content: dict):
        """ Initialize from serialized content. """
        pass


In [6]:
#Load_results
import json

from nuscenes.eval.tracking.data_classes import TrackingConfig
from nuscenes.eval.detection.data_classes import DetectionConfig

with open('/home/albhaisi/Desktop/Presentation/for_automatic_test/results/tracking_nips_2023.json', 'r') as _f:
    cfg = TrackingConfig.deserialize(json.load(_f))
with open('/home/albhaisi/Desktop/Presentation/for_automatic_test/base_codes/Final_codes/detection_cvpr_2023.json', 'r') as _f:
    cfg2 = DetectionConfig.deserialize(json.load(_f))

def load_prediction(result_path: str, max_boxes_per_sample: int, box_cls, verbose: bool = False) \
        -> Tuple[EvalBoxes, Dict]:
    """
    Loads object predictions from file.
    :param result_path: Path to the .json result file provided by the user.
    :param max_boxes_per_sample: Maximim number of boxes allowed per sample.
    :param box_cls: Type of box to load, e.g. DetectionBox or TrackingBox.
    :param verbose: Whether to print messages to stdout.
    :return: The deserialized results and meta data.
    """

    # Load from file and check that the format is correct.
    with open(result_path) as f:
        data = json.load(f)
    assert 'results' in data, 'Error: No field `results` in result file. Please note that the result format changed.' \
                              'See https://www.nuscenes.org/object-detection for more information.'

    # Deserialize results and get meta data.
    all_results = EvalBoxes.deserialize(data['results'], box_cls)
    
    meta = data['meta']
    if verbose:
        print("Loaded results from {}. Found detections for {} samples."
              .format(result_path, len(all_results.sample_tokens)))

    # Check that each sample has no more than x predicted boxes.
    for sample_token in all_results.sample_tokens:
        assert len(all_results.boxes[sample_token]) <= max_boxes_per_sample, \
            "Error: Only <= %d boxes per sample allowed!" % max_boxes_per_sample

    return all_results, meta


pred_boxes, meta = load_prediction(result_path, cfg.max_boxes_per_sample, TrackingBox,
                                                verbose=verbose)
print(load_prediction(result_path,cfg.max_boxes_per_sample,TrackingBox))

pred_boxes2, meta2 = load_prediction(result_path, cfg.max_boxes_per_sample, DetectionBox,
                                                     verbose=verbose)
gt_boxes2 = load_gt(nusc, eval_set, DetectionBox, verbose=verbose)

#assert set(pred_boxes2.sample_tokens) == set(gt_boxes2.sample_tokens), \
#            "Samples in split don't match samples in predicted tracks."


Loaded results from /home/albhaisi/Desktop/senarios/final_results/3_straight_toward_ego/Pillar02.json. Found detections for 31 samples.
(EvalBoxes with 496 boxes across 31 samples, {'use_camera': False, 'use_lidar': True, 'use_radar': False, 'use_map': False, 'use_external': False})
Loaded results from /home/albhaisi/Desktop/senarios/final_results/3_straight_toward_ego/Pillar02.json. Found detections for 31 samples.
Loading annotations for val split from nuScenes version: v1.0-trainval


100%|██████████| 31/31 [00:00<00:00, 599.86it/s]

Loaded ground truth annotations for 31 samples.


In [7]:
from nuscenes.eval.common.loaders import add_center_dist,filter_eval_boxes
from nuscenes.eval.tracking.loaders import create_tracks 
from nuscenes.eval.detection.data_classes import DetectionMetricDataList
from nuscenes.eval.detection.algo import accumulate, calc_ap, calc_tp
from nuscenes.eval.detection.constants import TP_METRICS
from nuscenes.eval.detection.render import summary_plot, class_pr_curve, class_tp_curve, dist_pr_curve, visualize_sample

   # Add center distances.
pred_boxes = add_center_dist(nusc, pred_boxes)
gt_boxes = add_center_dist(nusc, gt_boxes)

# -----------------------------------
# Step 1: Accumulate metric data for all classes and distance thresholds.
# -----------------------------------
if verbose:
    print('Accumulating metric data...')
metric_data_list = DetectionMetricDataList()

for class_name in cfg2.class_names:
    for dist_th in cfg2.dist_ths:
        continue


        
    
    


Accumulating metric data...


In [8]:
#Initialization

from nuscenes.eval.common.loaders import add_center_dist,filter_eval_boxes
from nuscenes.eval.tracking.loaders import create_tracks

# Add center distances.
pred_boxes = add_center_dist(nusc, pred_boxes)
gt_boxes = add_center_dist(nusc, gt_boxes)

print(pred_boxes)
print(gt_boxes)

# Filter boxes (distance, points per box, etc.).
if verbose:
            print('Filtering tracks')
            pred_boxes = filter_eval_boxes(nusc, pred_boxes, cfg.class_range, verbose=verbose)
if verbose:
            print('Filtering ground truth tracks')
            gt_boxes = filter_eval_boxes(nusc, gt_boxes, cfg.class_range, verbose=verbose)

            sample_tokens = gt_boxes.sample_tokens

            # Convert boxes to tracks format.
            tracks_gt = create_tracks(gt_boxes, nusc, eval_set, gt=True)
            tracks_pred = create_tracks(pred_boxes, nusc, eval_set, gt=False)



EvalBoxes with 496 boxes across 31 samples
EvalBoxes with 893 boxes across 31 samples
Filtering tracks
=> Original number of boxes: 496
=> After distance based filtering: 421
=> After LIDAR and RADAR points based filtering: 421
=> After bike rack filtering: 421
Filtering ground truth tracks
=> Original number of boxes: 893
=> After distance based filtering: 495
=> After LIDAR and RADAR points based filtering: 420
=> After bike rack filtering: 420


In [9]:
#Detection
class_name='car'
npos = len([1 for gt_boxes2 in gt_boxes.all if gt_boxes2.tracking_name == 'car'])
if verbose:
        print("Found {} GT of class {} out of {} total across {} samples.".
              format(npos, class_name, len(gt_boxes.all), len(gt_boxes.sample_tokens)))
# Organize the predictions in a single list.
pred_boxes_list = [box for box in pred_boxes2.all if box.tracking_name == 'car']
pred_confs = [box.tracking_score for box in pred_boxes_list]
print(pred_confs)
if verbose:
        print("Found {} PRED of class {} out of {} total across {} samples.".
              format(len(pred_confs), class_name, len(pred_boxes2.all), len(pred_boxes2.sample_tokens)))
                            
# Sort by confidence.
sortind = [i for (v, i) in sorted((v, i) for (i, v) in enumerate(pred_confs))][::-1]
#Sort based on instance annotations
sortind2 = [i for (v, i) in ((v, i) for (i, v) in enumerate(pred_confs))]
#print(sortind2)
# Do the actual matching.
tp = []  # Accumulator of true positives.
fp = []  # Accumulator of false positives.
conf = []  # Accumulator of confidences.


# match_data holds the extra metrics we calculate for each match.
match_data = {'trans_err': [],
                  'vel_err': [],
                  'scale_err': [],
                  'orient_err': [],
                  'attr_err': [],
                  'IOU': [],
                  'conf': []}


Found 474 GT of class car out of 474 total across 31 samples.
[0.2336622178554535, 0.2336622178554535, 0.2374500334262848, 0.2336622178554535, 0.2374500334262848, 0.6251834630966187, 0.2336622178554535, 0.2374500334262848, 0.6251834630966187, 0.6389774084091187, 0.2336622178554535, 0.2374500334262848, 0.6251834630966187, 0.6389774084091187, 0.6922845244407654, 0.2336622178554535, 0.2374500334262848, 0.6251834630966187, 0.6389774084091187, 0.6922845244407654, 0.6075785160064697, 0.2336622178554535, 0.2374500334262848, 0.6251834630966187, 0.6389774084091187, 0.6922845244407654, 0.6075785160064697, 0.5507705807685852, 0.2336622178554535, 0.2374500334262848, 0.6251834630966187, 0.6389774084091187, 0.6922845244407654, 0.6075785160064697, 0.5507705807685852, 0.6230815649032593, 0.2336622178554535, 0.2374500334262848, 0.6251834630966187, 0.6389774084091187, 0.6922845244407654, 0.6075785160064697, 0.5507705807685852, 0.6230815649032593, 0.7652725577354431, 0.2336622178554535, 0.237450033426284

In [10]:
from typing import Callable
from nuscenes.eval.common.utils import center_distance, scale_iou, yaw_diff, velocity_l2, attr_acc, cummean
from nuscenes.eval.detection.data_classes import DetectionMetricDataList


# ---------------------------------------------
# Match and accumulate match data.
# ---------------------------------------------
#All results sorted based on instance annotations

taken = set()  # Initially no gt bounding box is matched.
for ind in sortind2:
    pred_box = pred_boxes_list[ind]
    min_dist = np.inf
    match_gt_idx = None
    for gt_idx, gt_box in enumerate(gt_boxes[pred_box.sample_token]):
        # Find closest match among ground truth boxes
        if gt_box.tracking_name == class_name and not (pred_box.sample_token, gt_idx) in taken:
            this_distance = cfg.dist_fcn_callable(gt_box, pred_box)
            if this_distance < min_dist:
                min_dist = this_distance
                match_gt_idx = gt_idx
                # add reference to closest annotation
                #sample_annotation_token = gt_boxes2.sample_annotation_token

    # If the closest match is close enough according to threshold we have a match!
    is_match = min_dist < dist_th

    if is_match:
        taken.add((pred_box.sample_token, match_gt_idx))
        #  Update tp, fp and confs.
        tp.append(1)
        fp.append(0)
        conf.append(pred_box.tracking_score)
        # Since it is a match, update match data also.
        gt_box_match = gt_boxes[pred_box.sample_token][match_gt_idx]
        match_data['IOU'].append(scale_iou(gt_box_match, pred_box))
        match_data['scale_err'].append(1 - scale_iou(gt_box_match, pred_box))
        match_data['trans_err'].append(center_distance(gt_box_match, pred_box))
        period = np.pi if class_name == 'barrier' else 2 * np.pi
        match_data['orient_err'].append(yaw_diff(gt_box_match, pred_box, period=period))
        match_data['conf'].append(pred_box.tracking_score)
        #print(yaw_diff(gt_box_match, pred_box, period=period))
    
#test
print(match_data['IOU'])
print(match_data['orient_err'])
print(match_data['trans_err'])
print(match_data['conf'])
#print(match_data['vis'])
        

if verbose:
            print('Accumulating metric data...')
metric_data_list = DetectionMetricDataList()


[0.7607120059846387, 0.7800112187207622, 0.7837286575146237, 0.7873336310032519, 0.7784990293034264, 0.7616457797976314, 0.7730906994681138, 0.7887112745227776, 0.7929647161203985, 0.8076926328181601, 0.8219570714912865, 0.8187830131721165, 0.8182528060902118, 0.830941953337478, 0.8412289310733418, 0.8453409706832368, 0.8375696320671565, 0.8356379632539251, 0.8091689958966969, 0.8020072885390818, 0.8117387873006006, 0.8028154122852674, 0.7802813541858948, 0.7929839302952469, 0.798989854753833, 0.8008397604213413, 0.7825783582945993, 0.7816158909639616, 0.7842131985811382, 0.8148524924116989]
[0.059162935140030015, 0.0034181144184710632, 0.00033219461261513317, 0.009673631601942567, 0.011663316138205815, 0.031556644418904156, 0.01862981393160723, 0.016371084498848276, 0.0035647309841637664, 0.01917883524267161, 0.028288260244789143, 0.004165692978447488, 0.027165646047012437, 0.0420762439326996, 0.020312147632213673, 0.01745884741421655, 0.010004861842516366, 0.026491915284462486, 0.058

In [11]:
print(len([0.2374500334262848, 0.6251834630966187, 0.6389774084091187, 0.6922845244407654, 0.6075785160064697, 0.5507705807685852, 0.6230815649032593, 0.7652725577354431, 0.7028530836105347, 0.7705720663070679, 0.7259702086448669, 0.7842296361923218, 0.7694668769836426, 0.7976507544517517, 0.8406249284744263, 0.7897068858146667, 0.7305086255073547, 0.8173148036003113, 0.7590887546539307, 0.8752301335334778, 0.8019616603851318, 0.779859721660614, 0.7275107502937317, 0.8721888661384583, 0.8153125643730164, 0.7996419072151184, 0.7963605523109436, 0.7977954149246216, 0.8439611196517944, 0.8847833275794983]
))

30


: 

In [46]:

# ---------------------------------------------
# Calculate and interpolate precision and recall
# ---------------------------------------------
# Do the actual matching.
tp = []  # Accumulator of true positives.
fp = []  # Accumulator of false positives.
conf = []  # Accumulator of confidences.
# Accumulate.
tp = np.cumsum(tp).astype(float)
fp = np.cumsum(fp).astype(float)
conf = np.array(conf)

# Calculate precision and recall.
prec = tp / (fp + tp)
rec = tp / float(npos)




In [47]:
from nuscenes.eval.detection.data_classes import DetectionConfig, DetectionMetrics
import os

_label_aps = defaultdict(lambda: defaultdict(float))
metrics = DetectionMetrics(cfg2)

output_dir = '/home/albhaisi/Desktop/Presentation/for_automatic_test/base_codes/test'

def mean_dist_aps():
        """ Calculates the mean over distance thresholds for each label. """
        return {class_name: np.mean(list(d.values())) for class_name, d in _label_aps.items()}

def mean_ap(self) -> float:
        """ Calculates the mean AP by averaging over distance thresholds and classes. """
        return float(np.mean(list(mean_dist_aps.values())))


if verbose:
    print('Saving metrics to: %s' % output_dir)

metrics_summary = metrics.serialize()
metrics_summary['meta'] = meta.copy()

with open(os.path.join(output_dir, 'metrics_details.json'), 'w') as f:
            f.write('Final_metrics_calculation: \n \n')

            f.write('Confidence_scor: \n \n')
            json.dump(match_data['conf'], f, indent=2)

            f.write('IOU: \n \n')
            json.dump(match_data['IOU'], f, indent=2)

            f.write('Translation_error: \n \n')
            json.dump(match_data['trans_err'], f, indent=2)
            
            f.write('Orientation_error: \n \n')
            json.dump(match_data['orient_err'], f, indent=2)



Saving metrics to: /home/albhaisi/Desktop/Presentation/for_automatic_test/base_codes/test
